In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.2 on the current Colab Runtime...
2023-10-24 13:32:24 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.2-linux-x86_64.tar.gz [135859273/135859273] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.8

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


## Package Installation

In [12]:
import Pkg
Pkg.add("StochasticPrograms")
Pkg.add("HiGHS")
Pkg.add("Random")

Pkg.add("Distributions")

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


    Updating `~/.julia/environments/v1.9/Project.toml`
  [9a3f8284] + Random
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


## Package Import

In [1]:
using StochasticPrograms
using HiGHS

In [2]:
using Random, Distributions


In [3]:
using BenchmarkTools

### Complete Models

In [89]:
procurement_c = [1.0, 10.0, 100.0] # procurement cost
holding_c = 1.0 # holding cost
stockout_c = [[1.5*c, 5*c, 10*c] for c in procurement_c] # stock out cost

models = Vector{StochasticModel}()

for c in procurement_c
    stockout_c = [1.5*c, 5*c, 10*c]
    for v in stockout_c
        name = "newsvendor_c$(c)_v$(v)"

        @stochastic_model name begin
            @stage 1 begin
                @parameters begin
                    c = c
                end
                @decision(name, x >= 0)
                @constraint(name, x <= 10000)
                @objective(name, Min, x * c)
            end
            @stage 2 begin
                @parameters begin
                  h = holding_c
                  v = v
                end
                @uncertain ξ
                @recourse(name, y >= 0)
                @recourse(name, ξ >= w >= 0)

                @objective(name, Min, h * y + v*w)
                @constraint(name, y >= x-ξ)
                @constraint(name, w >= ξ-x)
            end
        end
        push!(models, name)
    end
end

## Scenarios and Distributions

In [5]:
Random.seed!(42)

TaskLocalRNG()

In [6]:
function generate_scenarios(distrib, max_val = 200)
    scenarios = [@scenario ξ = n probability = cdf(distrib, n) - cdf(distrib, n-1) for n in 1:max_val]
    return scenarios
end

generate_scenarios (generic function with 2 methods)

In [7]:
Unif = Uniform(0, 200)
Exp1 = truncated(Exponential(100), 0, 200)
Exp2 = truncated(Exponential(1e6), 0, 200)

N1 = truncated(Normal(100, 10), 0, 200)
N2 = truncated(Normal(100, 50), 0, 200)
N3 = truncated(Normal(100, 100), 0, 200)

Trian = truncated(TriangularDist(0, 200, 100), 0, 200)
distribs = [Unif, Exp1, Exp2, N1, N2, N3, Trian]


7-element Vector{Distribution{Univariate, Continuous}}:
 Uniform{Float64}(a=0.0, b=200.0)
 Truncated(Exponential{Float64}(θ=100.0); lower=0.0, upper=200.0)
 Truncated(Exponential{Float64}(θ=1.0e6); lower=0.0, upper=200.0)
 Truncated(Normal{Float64}(μ=100.0, σ=10.0); lower=0.0, upper=200.0)
 Truncated(Normal{Float64}(μ=100.0, σ=50.0); lower=0.0, upper=200.0)
 Truncated(Normal{Float64}(μ=100.0, σ=100.0); lower=0.0, upper=200.0)
 Truncated(TriangularDist{Float64}(a=0.0, b=200.0, c=100.0); lower=0.0, upper=200.0)

In [8]:
tot_scenarios = [generate_scenarios(d) for d in distribs]

7-element Vector{Vector{Scenario{NamedTuple{(:ξ,), Tuple{Int64}}}}}:
 [Scenario with probability 0.005
  ξ: 1, Scenario with probability 0.005
  ξ: 2, Scenario with probability 0.004999999999999999
  ξ: 3, Scenario with probability 0.005000000000000001
  ξ: 4, Scenario with probability 0.005000000000000001
  ξ: 5, Scenario with probability 0.0049999999999999975
  ξ: 6, Scenario with probability 0.0050000000000000044
  ξ: 7, Scenario with probability 0.0049999999999999975
  ξ: 8, Scenario with probability 0.0049999999999999975
  ξ: 9, Scenario with probability 0.0050000000000000044
  ξ: 10  …  Scenario with probability 0.0050000000000000044
  ξ: 191, Scenario with probability 0.0050000000000000044
  ξ: 192, Scenario with probability 0.0050000000000000044
  ξ: 193, Scenario with probability 0.0050000000000000044
  ξ: 194, Scenario with probability 0.0050000000000000044
  ξ: 195, Scenario with probability 0.0050000000000000044
  ξ: 196, Scenario with probability 0.0050000000000000044
  ξ:

In [11]:
typeof(models[1])

StochasticModel{2, Tuple{StageParameters{NamedTuple{(:c,), Tuple{Float64}}}, StageParameters{NamedTuple{(:h, :v), Tuple{Float64, Float64}}}}}

## L-Shaped Solution

In [ ]:
itl = instantiate(models[1], tot_scenarios[1], optimizer = LShaped.Optimizer)
set_optimizer_attribute(itl, MasterOptimizer(), HiGHS.Optimizer)
set_optimizer_attribute(itl, SubProblemOptimizer(), HiGHS.Optimizer)
optimize!(itl)
value(itl[1, :x])

In [90]:
opt_order_quantities = Array(ones(length(models),7))
objective_vals =Array(ones(length(models),7))

for i in 1:length(tot_scenarios)
    println("i\t", i)
    println("Distribution: ", distribs[i])
    for model_idx in 1:length(models)

        newsvendor = instantiate(models[model_idx], tot_scenarios[i], optimizer = HiGHS.Optimizer)
        # newsvendor = instantiate(models[model_idx], tot_scenarios[i], optimizer = LShaped.Optimizer)
        # set_optimizer_attribute(newsvendor, MasterOptimizer(), HiGHS.Optimizer)
        # set_optimizer_attribute(newsvendor, SubProblemOptimizer(), HiGHS.Optimizer)
        optimize!(newsvendor)
        println("TERMINATION", termination_status(newsvendor))

        opt_order_quantities[ model_idx, i] = value(newsvendor[1,:x])
        
        objective_vals[model_idx, i] = objective_value(newsvendor)


    end
end

In [93]:
# opt_order_quantities
objective_vals

9×7 Matrix{Float64}:
   140.5      98.9488    140.496    107.496    131.554    138.279    129.418
   233.83    196.471     233.828    122.311    198.81     224.099    191.632
   264.13    244.909     264.13     129.559    228.896    255.599    220.096
  1348.74    960.817    1348.7     1061.66    1259.88    1324.33    1241.48
  1867.73   1591.9      1867.71    1154.33    1669.25    1817.52    1623.35
  1985.15   1820.34     1985.15    1193.52    1818.81    1950.12    1762.73
 13394.3    9558.47    13393.9    10597.4    12517.3    13150.8    12337.3
 18113.6   15431.3     18113.4    11458.9    16304.8    17663.8    15874.4
 19130.5   17461.1     19130.5    11818.1    17650.3    18824.9    17125.5

### Probability Generation

In [17]:
function generate_probs(distrib, max_val=200)
    probability = [cdf(distrib, n) - cdf(distrib, n-1) for n in 1:max_val]
    return probability
end

generate_probs (generic function with 2 methods)

In [18]:
tot_probs = [generate_probs(d) for d in distribs]

7-element Vector{Vector{Float64}}:
 [0.005, 0.005, 0.004999999999999999, 0.005000000000000001, 0.005000000000000001, 0.0049999999999999975, 0.0050000000000000044, 0.0049999999999999975, 0.0049999999999999975, 0.0050000000000000044  …  0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044, 0.0050000000000000044]
 [0.011507542817379443, 0.011393040853207947, 0.011279678202616005, 0.011167443529244099, 0.01105632560953134, 0.010946313331593162, 0.010837395694110144, 0.010729561805227661, 0.010622800881467082, 0.010517102246646995  …  0.0017211672898407304, 0.0017040413891614703, 0.001687085894040985, 0.001670299108915696, 0.0016536793550935291, 0.0016372249705846054, 0.001620934309937594, 0.0016048057440717356, 0.001588837660118081, 0.0015730284612541778]
 [0.005000497516417494, 0.005000492515922476, 0.005000487515432464, 0.005000482514947

## EGD

In [85]:
function generate_models_fixed(real_idx, opt_order_quantities)

    procurement_c = [1.0, 10.0, 100.0] # procurement cost
    holding_c = 1.0 # holding cost
    stockout_c = [[1.5*c, 5*c, 10*c] for c in procurement_c] # stock out cost

    models_fixed = Vector{Vector{StochasticModel}}()

    for i in 1:length(tot_scenarios)
        
        model_idx = 1
        if i == real_idx
            continue
        end
        curr_models_fixed = Vector{StochasticModel}()

        for c in procurement_c
            stockout_c = [1.5*c, 5*c, 10*c]
            for v in stockout_c
                
                name = "newsvendor_c$(c)_v$(v)_d$(i)"

                @stochastic_model name begin
                    @stage 1 begin
                        @parameters begin
                            c = c
                            x = opt_order_quantities[model_idx, i]
                        end
                        @objective(name, Min, x * c)
                    end
                    @stage 2 begin
                        @parameters begin
                        h = holding_c
                        v = v
                        end
                        @uncertain ξ
                        @recourse(name, y >= 0)
                        @recourse(name, ξ >= w >= 0)

                        @objective(name, Min, h * y + v*w)
                        @constraint(name, y >= x-ξ)
                        @constraint(name, w >= ξ-x)
                    end
                end

                
                # models_fixed[model_idx, i] = name
                push!(curr_models_fixed, name)
                model_idx += 1
            end
        end
        push!(models_fixed, curr_models_fixed)
    end
    return models_fixed

end

generate_models_fixed (generic function with 2 methods)

In [ ]:
# function calculate_egd(real_idx, opt_order_quantities, models, tot_scenarios, distribs)
real_idx = 1
    opt_order_egd = Array(ones(length(models), 6))
    objective_egd =Array(ones(length(models), 6))
    models_fixed = Array(ones(length(models), 6))


    models_fixed = generate_models_fixed(real_idx, opt_order_quantities)
   
    for i in 1:length(models_fixed)
        curr_distrib_models = models_fixed[i]
        for model_idx in 1:length(curr_distrib_models)
            newsvendor = instantiate(curr_distrib_models[model_idx], tot_scenarios[real_idx], optimizer = HiGHS.Optimizer)
            # newsvendor = instantiate(models[model_idx], tot_scenarios[real_idx], optimizer = LShaped.Optimizer)
            
            println(opt_order_quantities[model_idx, i])
            # set_start_value(dvar::DecisionVariable, scenario_index::Integer, value::Number)
            # JuMP.fix(newsvendor[1, :x], opt_order_quantities[model_idx, i])
            optimize!(newsvendor)

            opt_order_egd[model_idx, i] = value(newsvendor[1,:x])
            
            objective_egd[model_idx, i] = objective_value(newsvendor)
        end
    end
    # return opt_order_egd, objective_egd
# end

In [154]:
function calculate_egd(real_idx, opt_order_quantities, models, tot_scenarios, distribs)
# real_idx = 2
    opt_order_egd = Array(ones(length(models), 6))
    objective_egd =Array(ones(length(models), 6))

    for i in 1:length(tot_scenarios)

        println("i\t", i)
        println("Distribution: ", distribs[i])

        if i == real_idx 
            println("Skipping ", i)
            continue
        end

        for model_idx in 1:length(models)

            # newsvendor = instantiate(models[model_idx], tot_scenarios[real_idx], optimizer = HiGHS.Optimizer)
            newsvendor = instantiate(models[model_idx], tot_scenarios[real_idx], optimizer = LShaped.Optimizer)
            set_optimizer_attribute(newsvendor, MasterOptimizer(), HiGHS.Optimizer)
            set_optimizer_attribute(newsvendor, SubProblemOptimizer(), HiGHS.Optimizer)

            JuMP.fix(newsvendor[1, :x], opt_order_quantities[model_idx, i])
            optimize!(newsvendor)
            
            if real_idx < i
                opt_order_egd[model_idx, i-1] = value(newsvendor[1,:x])
                objective_egd[model_idx, i-1] = objective_value(newsvendor)
            else
                opt_order_egd[model_idx, i] = value(newsvendor[1,:x])
                objective_egd[model_idx, i] = objective_value(newsvendor)
            end
        end

    end
    return opt_order_egd, objective_egd
end

calculate_egd (generic function with 1 method)

## VRD

In [166]:
function calculate_metrics(egd, rpr)
    n, m = size(egd)

    vrd = Array(ones(n+1,m))
    pb = Array(ones(n+1, m))

    for i = 1:n
        for j = 1:m
            vrd[i, j] = abs(egd[i, j] - rpr[i])
            pb[i, j] = abs(egd[i, j] / rpr[i] -1) * 100
        end
    end

    for j = 1:m
        vrd[n+1, j] = mean(vrd[1:n+1, j])
        pb[n+1, j] = mean(pb[1:n+1, j])
    end

    return vrd, pb
end

calculate_metrics (generic function with 2 methods)

## Uniform Distribution Results

In [133]:
# function generate_metrics_unit(i, opt_order_quantities, objective_vals, models, tot_scenarios, distribs, calc_vrd=false)
i =1
calc_vrd = true 
opt_order_egd, objective_egd = calculate_egd(i, opt_order_quantities, models, tot_scenarios, distribs)
vrd1, pb1 = calculate_metrics(objective_egd,  objective_vals)

println("Distribution: ", distribs[i])
if calc_vrd
    println("VRD: ", vrd1)
end
println("PB: ", pb1)

In [134]:
vrd1

10×6 Matrix{Float64}:
    2.8875  0.0             16.575     2.625   0.2625     3.45
   34.32    2.84217e-14     12.47      2.47    0.35       3.3
   46.535   0.0             80.595    12.285   1.035     16.02
   38.44    4.54747e-13     43.56      7.84    1.0       11.56
  239.725   2.27374e-13    310.66     47.785   5.145     63.745
  238.235   0.0           1090.48    108.36    8.235    154.25
  404.25    3.63798e-12    323.495    66.495   6.87      88.05
 2228.41    3.63798e-12   3280.83    510.05   47.805    672.98
 2151.08    0.0          11341.1    1137.26   70.575   1479.9
  538.488   0.1           1650.08    189.617  14.2278   249.425

In [135]:
pb1

10×6 Matrix{Float64}:
  2.05516   0.0          11.7972   1.86833   0.186833   2.45552
 14.6773    2.22045e-14   5.33293  1.05632   0.149681   1.41128
 17.6182    0.0          30.5134   4.65112   0.391852   6.0652
  2.85007   4.44089e-14   3.22968  0.581283  0.0741433  0.857096
 12.8351    2.22045e-14  16.633    2.55845   0.275468   3.41297
 12.0008    0.0          54.932    5.45852   0.414829   7.77017
  3.01807   2.22045e-14   2.41517  0.496442  0.0512905  0.657369
 12.3024   -2.22045e-14  18.1125   2.81584   0.263918   3.71533
 11.2442    0.0          59.2826   5.94471   0.368913   7.73579
  8.96014   0.1          20.3248   2.6431    0.317693   3.50807

## Exponential 1

In [155]:
i = 2
opt_order_egd, objective_egd = calculate_egd(i, opt_order_quantities, models, tot_scenarios, distribs)

([41.0 40.0 … 47.0 64.0; 134.0 134.0 … 129.0 119.0; … ; 160.0 160.0 … 154.0 137.0; 180.0 180.0 … 175.0 156.0], [104.10335716340639 103.65015545535468 … 107.22088912722208 119.48060988944113; 225.0482703079261 225.0482703079261 … 219.7152268742256 210.51382615328447; … ; 16830.164613772195 16830.164613772195 … 16480.671401553485 15743.915860849549; 18463.50156401682 18463.50156401682 … 18161.17985902615 17496.003210291572])

In [167]:
vrd2, pb2 = calculate_metrics(objective_egd,  objective_vals)

println("Distribution: ", distribs[i])
println("PB: ", pb2)

In [168]:
pb2

10×6 Matrix{Float64}:
  5.32562   0.721441   5.1121    2.72722   7.84609   6.4492
 15.7908    5.48668   15.7908   15.9235   17.9985   17.012
 10.4936   12.6054    10.4936    3.31059  16.5749   13.0838
  6.75742   0.525883   6.75742   6.59757   7.92289   7.35739
  9.17013   8.78692    9.17013   5.75643  12.9957   10.9302
  5.03409  10.7847     5.03409  14.3571   11.0042    7.43813
  6.93762   0.562097   6.93762   6.92376   7.87522   7.37256
  8.27827   8.87519    8.27827   4.46451  12.2845   10.0513
  4.74323  10.2286     4.74323  17.5537   10.3489    6.68838
  7.35308   5.95769    7.33173   7.86143  10.5851    8.73829

## Exponential 2 

In [169]:
i = 3
opt_order_egd, objective_egd = calculate_egd(i, opt_order_quantities, models, tot_scenarios, distribs)
vrd2, pb3 = calculate_metrics(objective_egd,  objective_vals)

println("Distribution: ", distribs[i])
println("PB: ", pb3)

In [170]:
vrd2, pb3 = calculate_metrics(objective_egd,  objective_vals)

println("Distribution: ", distribs[i])
println("PB: ", pb3)

In [171]:
pb3

10×6 Matrix{Float64}:
 0.00292756    2.05174  11.7962   1.86607   0.184087   2.45337
 0.000782336  14.6736    5.33037  1.05477   0.148609   1.4096
 4.39165e-5   17.6148   30.5087   4.6496    0.391501   6.06342
 0.00278997    2.84662   3.22782  0.578869  0.0714834  0.854767
 0.000925799  12.8316   16.6292   2.55652   0.27425    3.41086
 0.000377598  11.9981   54.9253   5.45671   0.414124   7.76802
 0.0027628     3.01462   2.41319  0.494019  0.0486292  0.655
 0.000962493  12.299    18.1085   2.81384   0.262673   3.71315
 0.000494428  11.2415   59.2755   5.94278   0.368152   7.73361
 0.101207      8.95717  20.3215   2.64132   0.316351   3.50618

## Normal 1


In [158]:
i = 4
opt_order_egd, objective_egd = calculate_egd(i, opt_order_quantities, models, tot_scenarios, distribs)
vrd2, pb4 = calculate_metrics(objective_egd,  objective_vals)

println("Distribution: ", distribs[i])
println("PB: ", pb4)

In [173]:
vrd2, pb4 = calculate_metrics(objective_egd,  objective_vals)
pb4

10×6 Matrix{Float64}:
 0.00292756    2.05174  11.7962   1.86607   0.184087   2.45337
 0.000782336  14.6736    5.33037  1.05477   0.148609   1.4096
 4.39165e-5   17.6148   30.5087   4.6496    0.391501   6.06342
 0.00278997    2.84662   3.22782  0.578869  0.0714834  0.854767
 0.000925799  12.8316   16.6292   2.55652   0.27425    3.41086
 0.000377598  11.9981   54.9253   5.45671   0.414124   7.76802
 0.0027628     3.01462   2.41319  0.494019  0.0486292  0.655
 0.000962493  12.299    18.1085   2.81384   0.262673   3.71315
 0.000494428  11.2415   59.2755   5.94278   0.368152   7.73361
 0.101207      8.95717  20.3215   2.64132   0.316351   3.50618

## Normal 2

In [159]:
i = 5
opt_order_egd, objective_egd = calculate_egd(i, opt_order_quantities, models, tot_scenarios, distribs)
vrd2, pb5 = calculate_metrics(objective_egd,  objective_vals)

println("Distribution: ", distribs[i])
println("PB: ", pb5)

## Normal 3

In [160]:
i = 6
opt_order_egd, objective_egd = calculate_egd(i, opt_order_quantities, models, tot_scenarios, distribs)


In [174]:
vrd2, pb6 = calculate_metrics(objective_egd,  objective_vals)
pb6

10×6 Matrix{Float64}:
 0.00292756    2.05174  11.7962   1.86607   0.184087   2.45337
 0.000782336  14.6736    5.33037  1.05477   0.148609   1.4096
 4.39165e-5   17.6148   30.5087   4.6496    0.391501   6.06342
 0.00278997    2.84662   3.22782  0.578869  0.0714834  0.854767
 0.000925799  12.8316   16.6292   2.55652   0.27425    3.41086
 0.000377598  11.9981   54.9253   5.45671   0.414124   7.76802
 0.0027628     3.01462   2.41319  0.494019  0.0486292  0.655
 0.000962493  12.299    18.1085   2.81384   0.262673   3.71315
 0.000494428  11.2415   59.2755   5.94278   0.368152   7.73361
 0.101207      8.95717  20.3215   2.64132   0.316351   3.50618

## Triangular

In [161]:
i = 7
opt_order_egd, objective_egd = calculate_egd(i, opt_order_quantities, models, tot_scenarios, distribs)

In [176]:
vrd2, pb7 = calculate_metrics(objective_egd,  objective_vals)
pb7

10×6 Matrix{Float64}:
 0.00292756    2.05174  11.7962   1.86607   0.184087   2.45337
 0.000782336  14.6736    5.33037  1.05477   0.148609   1.4096
 4.39165e-5   17.6148   30.5087   4.6496    0.391501   6.06342
 0.00278997    2.84662   3.22782  0.578869  0.0714834  0.854767
 0.000925799  12.8316   16.6292   2.55652   0.27425    3.41086
 0.000377598  11.9981   54.9253   5.45671   0.414124   7.76802
 0.0027628     3.01462   2.41319  0.494019  0.0486292  0.655
 0.000962493  12.299    18.1085   2.81384   0.262673   3.71315
 0.000494428  11.2415   59.2755   5.94278   0.368152   7.73361
 0.101207      8.95717  20.3215   2.64132   0.316351   3.50618